In [ ]:
!pip install sentence-transformers -q

In [ ]:
! pip install pandas  -q

In [ ]:
!git clone https://github.com/solumnsilence/text-classifier.git --branch dev
%cd text-classifier

fatal: destination path 'text-classifier' already exists and is not an empty directory.
/content/text-classifier


In [ ]:
from sentence_transformers import SentenceTransformer

model_st = SentenceTransformer('cointegrated/rubert-tiny2')

def get_embedding(text):
  return model_st.encode(text)

In [ ]:
import pandas as pd

train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

train_data.shape, test_data.shape

((38740, 3), (26260, 2))

In [ ]:
from tqdm import tqdm

features = []
targets = train_data['category'].tolist()

for i in tqdm(range(train_data.shape[0])):
  features.append(get_embedding(train_data.loc[i, 'text']).tolist())


100%|██████████| 38740/38740 [15:23<00:00, 41.97it/s]


In [ ]:
!pip install scikit-learn -q

In [ ]:
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

X = features
y = targets

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)


classifier = KNeighborsClassifier(3).fit(X_train, y_train)

In [ ]:
sum(classifier.predict(X_test)==y_test) / len(y_test)

0.6038203407330924

# Результаты исследования:

Я использовал предобученную нейронную сеть bert для русккого языка для получения фичей из текста и обучал алгоритмы классификации. В качестве feature extractor тестировал модели [rubert-tiny2](https://huggingface.co/cointegrated/) и [rubert-base-cased](https://huggingface.co/DeepPavlov/rubert-base-cased/) с алгоритмами knn и random forest.
После подбора гиперпараметров к алгоритмам классификации, лучшая точность с feature extractor rubert-tiny2 было 65%, а с rubert-base-cased 75%. Был выбран второй feature extractor и knn алгоритм т.к. random forest не давал значимого улучшения.

# Возможные улучшения

1. Нужно больше погружаться в домен и делать препроцессинг текста. (К примеру удалить все названия и имена из предложений)
2. Дообучить нейронку на текущей задаче
3. Попробовать другие feature extractors, однако нужно учитывать ограничения системы и её нагрузку
4. Лучший подбор гиперпараметров к классификаторам может дать небольшой прирост в точности.

P.S. часть ноутбука не сохранилась где тренировалась моделька, используемая в приложении